In [1]:
import geopandas as gpd
from fetch_data import make_dict_from_cols
import os
import json
import shapely
import numpy as np
import pandas as pd

In [2]:
geo_stations = gpd.read_file('./data/geojson/Amtrak_Project_Stations.geojson')

In [3]:
amtrak_stations = ['BOS', 'BBY', 'RTE', 'PVD', 'KIN', 'NLC', 'NHV', 'STM',
                   'NYP', 'NWK', 'TRE', 'PHL', 'WIL', 'BAL', 'BWI', 'NCR', 'WAS']

mile_markers = {'Northbound Mile': {station: None for station in amtrak_stations} , 'Southbound Mile': {station: None for station in amtrak_stations} } 

SB = [0,1,11,43,70,105,156,195,231,241,289,322,347,416,427,448,457]

NB = [457, 456, 446, 414,387,352,301,262,226,216,168,135,110,41,30,9,0]

for station, NB_mile, SB_mile in zip(amtrak_stations, NB, SB):
    mile_markers['Northbound Mile'][station] = NB_mile
    mile_markers['Southbound Mile'][station] = SB_mile


mile_cols = pd.DataFrame.from_dict(mile_markers, orient='columns')
mile_cols.to_csv('./data/geojson/mile_markers.csv', index=False)

In [4]:
stncodes = pd.Series(geo_stations.STNCODE, name='STNCODE')

lons = pd.Series(geo_stations.geometry.x, name='LON')
lats =  pd.Series(geo_stations.geometry.y, name='LAT')
#geo_stations['STNCODE'].values

gdf_lonlat = pd.DataFrame([lons, lats]).T
gdf_lonlat.set_index(geo_stations.STNCODE, inplace=True)

#gdf_lats = pd.Series(, name='Latitude', index=geo_stations.STNCODE)

#gdf_lats.index
gdf_lonlat

,LON,LAT
STNCODE,,
NYP,-73.994459,40.750327
WAS,-77.006422,38.896993
PHL,-75.181041,39.955615
BOS,-71.055304,42.352311
BAL,-76.615688,39.307302
PVD,-71.413478,41.829490
WIL,-75.551095,39.737263
BWI,-76.694300,39.192362
NWK,-74.164750,40.734706


In [5]:
gdf = pd.DataFrame(geo_stations[['STNCODE', 'STNNAME', 'CITY2', 'STATE']])

gdf = gdf.set_index(gdf['STNCODE']).drop('STNCODE', axis=1)
gdf = gdf.join(gdf_lonlat, how='inner')
gdf = gdf.join(mile_cols, how='inner')
gdf = gdf.loc[amtrak_stations]
gdf

,STNNAME,CITY2,STATE,LON,LAT,Northbound Mile,Southbound Mile
BOS,"Boston (South Station), Massachusetts",Boston,MA,-71.055304,42.352311,457,0
BBY,"Boston (Back Bay), Massachusetts",Boston,MA,-71.075828,42.347317,456,1
RTE,"Westwood, Route 128 Station, Massachusetts",Route 128,MA,-71.147894,42.210242,446,11
PVD,"Providence, Rhode Island",Providence,RI,-71.413478,41.829490,414,43
KIN,"West Kingston, Rhode Island",Kingston,RI,-71.560597,41.483959,387,70
NLC,"New London, Connecticut",New London,CT,-72.093225,41.354267,352,105
NHV,"New Haven, Connecticut",New Haven,CT,-72.926670,41.297714,301,156
STM,"Stamford, Connecticut",Stamford,CT,-73.542160,41.047130,262,195
NYP,"New York (Penn Station), New York",New York,NY,-73.994459,40.750327,226,231
NWK,"Newark (Penn Station), New Jersey",Newark,NJ,-74.164750,40.734706,216,241


In [6]:
lat = gdf['LAT']
lon = gdf['LON']
stations = gdf.index
geoloc_dict = {station: {'x': None, 'y': None, 'prev': None, 'next': None, 'path2prev': None, 'path2next': None} for station in stations}

prev_stat = None
for i in range(stations.shape[0]):
    print(station)
    station = stations[i]
    geoloc_dict[station]['x'] = lon.iloc[i]
    geoloc_dict[station]['y'] = lat.iloc[i]
    geoloc_dict[station]['prev'] = prev_stat
    if station != 'WAS':
        geoloc_dict[station]['next'] = stations[i+1]
    prev_stat = station
    print(station,  lon.iloc[i], lat.iloc[i], 'prev:', geoloc_dict[station]['prev'], 'next:', geoloc_dict[station]['next'])


WAS
BOS -71.05530399962942 42.35231100012763 prev: None next: BBY
BOS
BBY -71.07582800026684 42.34731700021211 prev: BOS next: RTE
BBY
RTE -71.14789400005567 42.21024199995967 prev: BBY next: PVD
RTE
PVD -71.4134779996708 41.829490000162025 prev: RTE next: KIN
PVD
KIN -71.5605969999761 41.483959000033835 prev: PVD next: NLC
KIN
NLC -72.09322499959666 41.35426700011465 prev: KIN next: NHV
NLC
NHV -72.92666999955892 41.29771399982176 prev: NLC next: STM
NHV
STM -73.54215999966355 41.047130000267344 prev: NHV next: NYP
STM
NYP -73.99445899996624 40.75032699989306 prev: STM next: NWK
NYP
NWK -74.16474999990626 40.73470599995154 prev: NYP next: TRE
NWK
TRE -74.75443999967372 40.21901100002319 prev: NWK next: PHL
TRE
PHL -75.18104099974647 39.95561500001248 prev: TRE next: WIL
PHL
WIL -75.55109500023144 39.737262999896686 prev: PHL next: BAL
WIL
BAL -76.61568799967372 39.30730200018028 prev: WIL next: BWI
BAL
BWI -76.69429999958173 39.19236199986559 prev: BAL next: NCR
BWI
NCR -76.8714940000

In [7]:
geo_df = gpd.read_file('./data/geojson/Amtrak_Project_Routes.geojson')

lats = []
lons = []
names = []

In [8]:
lats = []
lons = []
for feature in geo_df.geometry:
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        
print(np.min(lats), np.max(lats))
print(np.min(lons), np.max(lons))

37.023024 42.347551
-77.507591 -71.075149
